## Transfer Raw Scene Viewing Encoding Files to USB

This script is adjusted for retrieval blocks. It is currently not in use because the computer for the post scan task uses powershell version 1 which is not compatible.

In [ ]:
$sourceBase = "C:\Documents and Settings\UT\Desktop\Olsen Lab\Final Deployed Tasks Retrieval Aug 29"
$destinationBase = "E:\SceneViewing_DAY3"
$currentDate = Get-Date
$blocks = 1..3
$missingBlocks = @()
$dirInfo = @()

foreach ($block in $blocks) {
    $sourcePath = Join-Path -Path $sourceBase -ChildPath ("Last Block $block Retrieval Task Fixed_deploy\results")
    $rawdata = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }

    if ($rawdata.Count -eq 0) {
        $missingBlocks += $block
        continue
    } else {
        foreach ($dir in $rawdata) {
            $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
            $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)
            $dirSizeMBStr = $dirSizeMB.ToString() + "M"
            $dirInfo += "[$($dir.Name): $($dirSizeMBStr)]"
        }
    }
}

$dirInfoStr = $dirInfo -join ", "
$userInput = Read-Host "Do you want to move the following directories to their respective folders in the USB? Directories: $dirInfoStr [Y/N]"

if ($userInput -eq "Y") {
    if ($missingBlocks.Count -gt 0) {
        $missingBlockStr = $missingBlocks -join ", "
        $continueAnyway = Read-Host "Unexpected event: block(s) [$missingBlockStr] missing. Continue anyway? [Y/N]"
        if ($continueAnyway -ne "Y") {
            Write-Output "Operation aborted due to missing blocks."
            exit
        }
    }

    foreach ($block in $blocks) {
        if ($missingBlocks -contains $block) {
            Write-Output "Skipping block $block as it is missing."
            continue
        }

        $sourcePath = Join-Path -Path $sourceBase -ChildPath ("Last Block $block Retrieval Task Fixed_deploy\results")
        $destinationPath = Join-Path -Path $destinationBase -ChildPath ("Retrieval_Block$block")
        $rawdata = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }

        foreach ($dir in $rawdata) {
            $destPath = Join-Path -Path $destinationPath -ChildPath $dir.Name
            Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
            Write-Output "Copied $($dir.Name) to $($destinationPath)"
        }
    }
} else {
    Write-Output "Operation cancelled by user."
}
